In [12]:
import numpy as np
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.layers.wrappers import TimeDistributed
from keras.layers import Dense, Input
from keras.layers import GlobalMaxPooling1D, Bidirectional
from keras.models import Model, load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint
import pickle
from sklearn import metrics

In [9]:
bird_audio_with_label = pickle.load( open( "cat.pkl", "rb" ) )
X_bird = bird_audio_with_label["cat"]
Y_bird = np.array([[1,0]] * X_bird.shape[0])
print X_bird.shape
print Y_bird.shape

bed_audio_with_label = pickle.load( open( "dog.pkl", "rb" ) )
X_bed = bed_audio_with_label["dog"]
Y_bed = np.array([[0,1]] * X_bed.shape[0])
print X_bed.shape
print Y_bed.shape

X = np.vstack((X_bird, X_bed))
Y = np.vstack((Y_bird, Y_bed))
print X.shape
print Y.shape

(1733, 20, 50)
(1733, 2)
(1746, 20, 50)
(1746, 2)
(3479, 20, 50)
(3479, 2)


In [61]:
inp = Input(shape=(20,50 ))
x = (LSTM(50, return_sequences=True))(inp)
x = (LSTM(40, return_sequences=True))(x)
x = GlobalMaxPooling1D()(x)
x = Dropout(0.4)(x)

#x = Dense(20, activation = "sigmoid")(x)
#x = Dense(5, activation = "sigmoid")(x)

x = Dense(2, activation="softmax")(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        (None, 20, 50)            0         
_________________________________________________________________
lstm_27 (LSTM)               (None, 20, 50)            20200     
_________________________________________________________________
lstm_28 (LSTM)               (None, 20, 40)            14560     
_________________________________________________________________
global_max_pooling1d_12 (Glo (None, 40)                0         
_________________________________________________________________
dropout_12 (Dropout)         (None, 40)                0         
_________________________________________________________________
dense_12 (Dense)             (None, 2)                 82        
Total params: 34,842
Trainable params: 34,842
Non-trainable params: 0
_________________________________________________________________


In [62]:
file_path="RNN_val_loss_cat_dog.hdf5"
checkpoint = ModelCheckpoint(file_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
early = EarlyStopping(monitor="val_loss", mode="min", patience=20)
callbacks_list = [checkpoint, early] #early
model.fit(X, Y, validation_split=0.1, callbacks=callbacks_list, epochs=20, batch_size= 150)

Train on 3131 samples, validate on 348 samples
Epoch 1/20
3131/3131 [==============================] - 6s - loss: 0.6956 - acc: 0.5353 - val_loss: 0.8352 - val_acc: 0.0230
Epoch 2/20
3131/3131 [==============================] - 2s - loss: 0.6732 - acc: 0.5771 - val_loss: 0.7620 - val_acc: 0.2471
Epoch 3/20
3131/3131 [==============================] - 2s - loss: 0.6589 - acc: 0.6129 - val_loss: 0.7147 - val_acc: 0.4598
Epoch 4/20
3131/3131 [==============================] - 2s - loss: 0.6306 - acc: 0.6627 - val_loss: 0.8075 - val_acc: 0.3908
Epoch 5/20
3131/3131 [==============================] - 2s - loss: 0.5920 - acc: 0.6924 - val_loss: 0.8368 - val_acc: 0.5029
Epoch 6/20
3131/3131 [==============================] - 2s - loss: 0.5387 - acc: 0.7263 - val_loss: 0.7027 - val_acc: 0.6667
Epoch 7/20
3131/3131 [==============================] - 2s - loss: 0.5152 - acc: 0.7573 - val_loss: 0.4873 - val_acc: 0.7989
Epoch 8/20
3131/3131 [==============================] - 2s - loss: 0.4648 - ac

In [63]:
model = load_model(file_path)

In [64]:
a= model.predict(X)

In [65]:
a

array([[ 0.45884997,  0.54115009],
       [ 0.98628438,  0.01371567],
       [ 0.13852394,  0.86147612],
       ..., 
       [ 0.02537534,  0.97462463],
       [ 0.0184974 ,  0.98150259],
       [ 0.91128635,  0.08871356]], dtype=float32)

In [66]:
metrics.log_loss(Y,a)

0.28143203356117225